## Funciones utilizadas para Differentiable Augmentation

In [ ]:
import tensorflow as tf
# Functions for Differentiable Augmentation
def DiffAugment(x, policy='', channels_first=False):
    if policy:
        if channels_first:
            x = tf.transpose(x, [0, 2, 3, 1])
        for p in policy.split(','):
            for f in AUGMENT_FNS[p]:
                x = f(x)
        if channels_first:
            x = tf.transpose(x, [0, 3, 1, 2])
    return x

# Function to randomly change image brightness
def rand_brightness(x):
    magnitude = tf.random.uniform([tf.shape(x)[0], 1, 1, 1]) - 0.5
    x = x + magnitude
    return x

# Function to randomly change image saturation
def rand_saturation(x):
    magnitude = tf.random.uniform([tf.shape(x)[0], 1, 1, 1]) * 2
    x_mean = tf.reduce_mean(x, axis=3, keepdims=True)
    x = (x - x_mean) * magnitude + x_mean
    return x

# Function to randomly change image contrast
def rand_contrast(x):
    magnitude = tf.random.uniform([tf.shape(x)[0], 1, 1, 1]) + 0.5
    x_mean = tf.reduce_mean(x, axis=[1, 2, 3], keepdims=True)
    x = (x - x_mean) * magnitude + x_mean
    return x

# Function to randomly translate images
def rand_translation(x, ratio=0.125):
    batch_size = tf.shape(x)[0]
    image_size = tf.shape(x)[1:3]
    shift = tf.cast(tf.cast(image_size, tf.float32) * ratio + 0.5, tf.int32)
    translation_x = tf.random.uniform([batch_size, 1], -shift[0], shift[0] + 1, dtype=tf.int32)
    translation_y = tf.random.uniform([batch_size, 1], -shift[1], shift[1] + 1, dtype=tf.int32)
    grid_x = tf.clip_by_value(tf.expand_dims(tf.range(image_size[0], dtype=tf.int32), 0) + translation_x + 1, 0, image_size[0] + 1)
    grid_y = tf.clip_by_value(tf.expand_dims(tf.range(image_size[1], dtype=tf.int32), 0) + translation_y + 1, 0, image_size[1] + 1)
    x = tf.gather_nd(tf.pad(x, [[0, 0], [1, 1], [0, 0], [0, 0]]), tf.expand_dims(grid_x, -1), batch_dims=1)
    x = tf.transpose(tf.gather_nd(tf.pad(tf.transpose(x, [0, 2, 1, 3]), [[0, 0], [1, 1], [0, 0], [0, 0]]), tf.expand_dims(grid_y, -1), batch_dims=1), [0, 2, 1, 3])
    return x

# Function to randomly cut out parts of images
def rand_cutout(x, ratio=0.5):
    batch_size = tf.shape(x)[0]
    image_size = tf.shape(x)[1:3]
    cutout_size = tf.cast(tf.cast(image_size, tf.float32) * ratio + 0.5, tf.int32)
    offset_x = tf.random.uniform([tf.shape(x)[0], 1, 1], maxval=image_size[0] + (1 - cutout_size[0] % 2), dtype=tf.int32)
    offset_y = tf.random.uniform([tf.shape(x)[0], 1, 1], maxval=image_size[1] + (1 - cutout_size[1] % 2), dtype=tf.int32)
    grid_batch, grid_x, grid_y = tf.meshgrid(tf.range(batch_size, dtype=tf.int32), tf.range(cutout_size[0], dtype=tf.int32), tf.range(cutout_size[1], dtype=tf.int32), indexing='ij')
    cutout_grid = tf.stack([grid_batch, grid_x + offset_x - cutout_size[0] // 2, grid_y + offset_y - cutout_size[1] // 2], axis=-1)
    mask_shape = tf.stack([batch_size, image_size[0], image_size[1]])
    cutout_grid = tf.maximum(cutout_grid, 0)
    cutout_grid = tf.minimum(cutout_grid, tf.reshape(mask_shape - 1, [1, 1, 1, 3]))
    mask = tf.maximum(1 - tf.scatter_nd(cutout_grid, tf.ones([batch_size, cutout_size[0], cutout_size[1]], dtype=tf.float32), mask_shape), 0)
    x = x * tf.expand_dims(mask, axis=3)
    return x


AUGMENT_FNS = {
    'color': [rand_brightness, rand_saturation, rand_contrast],
    'translation': [rand_translation],
    'cutout': [rand_cutout],
}

## Ejecutar los modelos

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from keras.layers import Input, Conv2D, Dense, Flatten, BatchNormalization, LeakyReLU, ReLU, Reshape, Conv2DTranspose
import matplotlib.pyplot as plt
from tensorflow.keras import preprocessing, Sequential
import time
import io
from tqdm import tqdm

# Tensorboard writer is created
tensorboard= tf.summary.create_file_writer(logdir='logs/{}'.format("Cars{}".format(int(time.time()))))

# Directory where the images for training the network are located
directory="" # Format: "{directory}"

# Desired image size for the network to generate
img_size = 128

# Folder to save the trained model
trained_models_folder ="" # Format: "{directory}\ "

# Folder to save the images generated by the model
generated_images_folder="" # Format: "{directory}\ "

# Batch size
batch_size = 50

# Latent space dimension
latent_dim=100

# Loads the car dataset, normalizes and resizes its images
def get_loader(img_size):
    def train_preprocessing(x):
        x = tf.cast(x, tf.float32)
        x = x / 255.0  # normalization
        return x
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        directory,
        label_mode=None,
        batch_size=batch_size,
        shuffle=True,
        seed=123,
        image_size=(img_size, img_size),
    )
    datasetmapeado = dataset.map(lambda x: train_preprocessing(x))
    return datasetmapeado

dataset=get_loader(img_size)


# Builds the generator
def get_generator(latent_dim):
    # Initializes the generator weights with a normal distribution (stddev 0.02, mean 0.0) as in the original paper
    initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
    generator = Sequential(name='Generator')

    generator.add(Input(shape=(latent_dim,)))
    # Converts input to shape (8,8,1024)
    generator.add(Dense(8 * 8 * 1024))
    generator.add(Reshape((8, 8, 1024)))

    # Converts the vector to shape (16,16,512)
    generator.add(Conv2DTranspose(512, kernel_size=5, strides=2, padding='same',kernel_initializer=initializer, use_bias=False))
    generator.add(BatchNormalization(momentum=0.1, epsilon=0.8, center=1.0, scale=0.02))
    generator.add(ReLU())

    # Converts the vector to shape (32,32,256)
    generator.add(Conv2DTranspose(256, kernel_size=5, strides=2, padding='same',kernel_initializer=initializer, use_bias=False))
    generator.add(BatchNormalization(momentum=0.1, epsilon=0.8, center=1.0, scale=0.02))
    generator.add(ReLU())

    # Converts the vector to shape (64,64,128)
    generator.add(Conv2DTranspose(128, kernel_size=5, strides=2, padding='same',kernel_initializer=initializer, use_bias=False))
    generator.add(BatchNormalization(momentum=0.1, epsilon=0.8, center=1.0, scale=0.02))
    generator.add(ReLU())

    # Converts the vector to shape (128,128,64)
    generator.add(Conv2DTranspose(64, kernel_size=5, strides=2, padding='same',kernel_initializer=initializer, use_bias=False))
    generator.add(BatchNormalization(momentum=0.1, epsilon=0.8, center=1.0, scale=0.02))
    generator.add(ReLU())

    # Converts the vector to shape (128,128,3)
    generator.add(Conv2D(3, kernel_size=5, strides=1, padding='same',kernel_initializer=initializer, use_bias=False, activation='tanh'))

    return generator

# Builds the discriminator
def get_discriminator():
    # Initializes the discriminator weights with a normal distribution (stddev 0.02, mean 0.0) as in the original paper
    initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
    discriminator = Sequential(name='Discriminator')
    discriminator.add(Input(shape=(128, 128, 3)))

    # Converts the vector to shape (64,64,64)
    discriminator.add(Conv2D(64, kernel_size=5, strides=2, padding="same", kernel_initializer=initializer, use_bias=False))
    discriminator.add(BatchNormalization(momentum=0.1, epsilon=0.8, center=1.0, scale=0.02))
    discriminator.add(LeakyReLU(0.2))

    # Converts the vector to shape (32,32,128)
    discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding="same", kernel_initializer=initializer, use_bias=False))
    discriminator.add(BatchNormalization(momentum=0.1, epsilon=0.8, center=1.0, scale=0.02))
    discriminator.add(LeakyReLU(0.2))

    # Converts the vector to shape (16,16,256)
    discriminator.add(Conv2D(256, kernel_size=5, strides=2, padding="same",kernel_initializer=initializer, use_bias=False))
    discriminator.add(BatchNormalization(momentum=0.1, epsilon=0.8, center=1.0, scale=0.02))
    discriminator.add(LeakyReLU(0.2))

    # Converts the vector to shape (8,8,512)
    discriminator.add(Conv2D(512, kernel_size=5, strides=2, padding="same",kernel_initializer=initializer, use_bias=False))
    discriminator.add(BatchNormalization(momentum=0.1, epsilon=0.8, center=1.0, scale=0.02))
    discriminator.add(LeakyReLU(0.2))

    # Converts the vector to shape (4,4,1024)
    discriminator.add(Conv2D(1024, kernel_size=5, strides=2, padding="same",kernel_initializer=initializer, use_bias=False))
    discriminator.add(BatchNormalization(momentum=0.1, epsilon=0.8, center=1.0, scale=0.02))
    discriminator.add(LeakyReLU(0.2))
    
    # Converts the vector to shape (batch size, 1)
    discriminator.add(Flatten())
    discriminator.add(Dense(1, activation='sigmoid'))

    return discriminator

# Variables to hold the generator and discriminator models
generator=get_generator(latent_dim)
discriminator=get_discriminator()

# Loss function used
binary_cross_entropy = tf.keras.losses.BinaryCrossentropy()

# Generator loss function
def generator_loss(label, fake_output):
    gen_loss = binary_cross_entropy(label, fake_output)
    return gen_loss

# Discriminator loss function
def discriminator_loss(label, output):
    disc_loss = binary_cross_entropy(label, output)
    return disc_loss

# Initialize optimizers for generator and discriminator
gen_optimizer = disc_optimizer = Adam(0.0002, 0.5)

# Compile the models specifying their optimizers and loss function (Binary Cross Entropy)
generator.compile(gen_optimizer, loss=binary_cross_entropy)
discriminator.compile(disc_optimizer, loss=binary_cross_entropy)

# Print the summary of the generator and discriminator
generator.summary()
discriminator.summary()

# Returns images in a format that can be stored in Tensorboard
def plot_to_image(figure):
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(figure)
    buf.seek(0)
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)
    return image

# Creates a 5x5 grid with the received images
def image_grid(images):
    figure = plt.figure(figsize=(10,10))
    for i in range(images.shape[0]):
        img = preprocessing.image.array_to_img((images[i] + 1 / 2))
        plt.subplot(5, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(img)
    return figure

# Generates and saves the images produced by the generator in a local folder
def generate_and_save_images(folder, model, epoch, seed, dim=(5, 5), figsize=(10, 10)):
    generated_images = model(seed)
    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        img = preprocessing.image.array_to_img((generated_images[i] + 1 / 2))
        plt.imshow(img)
        plt.xticks([])
        plt.yticks([])
    plt.tight_layout()
    plt.savefig(folder + 'generated_image_epoch_%d.png' % epoch)
    plt.close()
    return generated_images

# Trains the network via train_step
@tf.function
def train_step(images):
    # Generates a vector of random images (noise)
    noise = tf.random.normal([batch_size, latent_dim])
    
    # Processes the batch images with Differentiable Augmentation
    images = DiffAugment(images, policy='color,translation,cutout')

    # Trains the discriminator with real images
    with tf.GradientTape() as disc_tapeReal:
        # Discriminator predictions for real images
        real_preds = discriminator(images, training=True)
        # Creates a vector of 1's multiplied by 0.9 to encourage learning, indicating real images
        unos = tf.ones_like(real_preds) * 0.9
        # Calculates discriminator loss with real images
        disc_lossReal = discriminator_loss(unos, real_preds)

    # Updates discriminator gradients
    gradients_discReal = disc_tapeReal.gradient(disc_lossReal, discriminator.trainable_variables)
    disc_optimizer.apply_gradients(zip(gradients_discReal, discriminator.trainable_variables))

    # Trains the discriminator with fake images
    with tf.GradientTape() as disc_tapeFake:
        # Generates fake images with the generator
        generated_images = generator(noise, training=True)
        # Processes the generated fake images with Differentiable Augmentation
        generated_images = DiffAugment(generated_images, policy='color,translation,cutout')
        # Discriminator predictions for fake images
        fake_preds = discriminator(generated_images, training=True)
        # Creates a vector of 0's indicating fake images
        ceros = tf.zeros_like(fake_preds)
        # Calculates discriminator loss with fake images
        disc_lossFake = discriminator_loss(ceros, fake_preds)

    # Updates discriminator gradients
    gradients_discFake = disc_tapeFake.gradient(disc_lossFake, discriminator.trainable_variables)
    disc_optimizer.apply_gradients(zip(gradients_discFake, discriminator.trainable_variables))

    # Trains the generator
    with tf.GradientTape() as gen_tape:
        # Generates fake images with the generator
        generated_images = generator(noise, training=True)
        # Processes the generated fake images with Differentiable Augmentation
        generated_images = DiffAugment(generated_images, policy='color,translation,cutout')
        # Discriminator predictions for fake images
        fake_preds = discriminator(generated_images, training=True)
        # "Tricks" the discriminator with a vector of 1's so it thinks the generated images are real and the generator learns
        unos = tf.ones_like(fake_preds)
        # Calculates generator loss
        gen_loss = generator_loss(unos, fake_preds)

    # Updates generator gradients
    gradients_gen = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gen_optimizer.apply_gradients(zip(gradients_gen, generator.trainable_variables))
    
    # Returns the sum of discriminator loss with fake and real images
    return disc_lossReal + disc_lossFake, gen_loss

# Creates a seed to always generate the same images and thus compare the network's evolution
tf.random.set_seed(347)
seed = tf.random.normal([25, latent_dim])

# Network training function
def train(dataset, epochs):
    # Generates images before training starts
    generate_and_save_images(generated_images_folder, generator, 0, seed)
    # Saves the discriminator and generator at epoch 0
    discriminator.save(trained_models_folder + "Discriminator_epoch_0")
    generator.save(trained_models_folder + "Generator_epoch_0")
    for epoch in range(epochs):
        print('Current training epoch {} (out of {}).'.format(epoch+1, epochs))
        # Loop that iterates over the dataset to train the network
        for image_batch in tqdm(dataset):
            disc_loss, gen_loss = train_step(image_batch)
        # Generates and saves fake images as the network evolves
        generated_images=generate_and_save_images(generated_images_folder, generator, epoch+1, seed)
        fig=image_grid(generated_images)
        with tensorboard.as_default():
            # Logs generator and discriminator loss and generated images to Tensorboard
            tf.summary.scalar('Loss discriminator', disc_loss.numpy(), step=epoch)
            tf.summary.scalar('Loss generator', gen_loss.numpy(), step=epoch)
            tf.summary.image('Generated images', plot_to_image(fig), step=epoch)

        # If the current epoch is a multiple of 10, save the generator and discriminator
        if epoch % 10 == 0:
            discriminator.save(trained_models_folder + "Discriminator_epoch_%d" % epoch)
            generator.save(trained_models_folder + "Generator_epoch_%d" % epoch)

    # In the last iteration, save the generator and discriminator and the last images produced by the generator
    generate_and_save_images(generated_images_folder, generator, epochs, seed)
    discriminator.save(trained_models_folder + "Discriminator_epoch_%d" % epochs)
    generator.save(trained_models_folder + "Generator_epoch_%d" % epochs)

# Calls the train function to start training
train(dataset, 200)

## Cargar los modelos

In [ ]:
# Specify the folder from which the trained models will be loaded
epoch = 0 # Specify the epoch of the model to load
generator = tf.keras.models.load_model(trained_models_folder + "Generator_epoch_%d" % epoch)
discriminator = tf.keras.models.load_model(trained_models_folder + "Discriminator_epoch_%d" % epoch)